In [ ]:
# %pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters

In [ ]:
%pip install -U langsmith openai
%pip install -qU langchain-text-splitters
%pip install tiktoken
%pip install -qU langchain_community pypdf
%pip install langchain-openai
%pip install chromadb
%pip install -U langgraph
!apt-get install -y graphviz graphviz-dev
!pip install pygraphviz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.42.2-6ubuntu0.1).
libgraphviz-dev is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# add you keys to run the notebook

import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"]=""
os.environ["LANGCHAIN_ENDPOINT"]=""
os.environ["LANGCHAIN_PROJECT"]=""


Document Ingestion

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/drive/MyDrive/helpdesk_usecase/IT_helpdesk.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

In [ ]:
docs[1].page_content

'o Enter your new password in both the "New password" and "Confirm new \npassword" fields. \no Click Next to finalize the reset. \n6. Confirmation: \no A confirmation message will indicate that your password has been successfully \nreset. \no You can now use your new password to log in to your account. \n3.2 Best Practices for Password Security \nTo maintain the security of your account, adhere to the following best practices: \n• Create Strong Passwords: \no Use a combination of uppercase and lowercase letters, numbers, and special \ncharacters. \no Avoid using easily guessable information such as birthdays, common words, or \nsequences (e.g., "password123" or "abcd1234"). \n• Unique Passwords: \no Do not reuse passwords across multiple accounts. Each account should have a \nunique password to prevent a compromise of one account leading to others being \ncompromised. \n• Regular Updates: \no Change your password periodically, at least every 90 days, to enhance security. \n• Use Multi-

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200,
)

In [ ]:
doc_splits =  text_splitter.split_documents(docs)

In [ ]:
doc_splits

[Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2025-04-09T08:50:17+05:00', 'author': 'Hizafa Nadeem', 'moddate': '2025-04-09T08:50:17+05:00', 'source': '/content/drive/MyDrive/helpdesk_usecase/IT_helpdesk.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}, page_content='IT Helpdesk Guidelines \n \nSection 3: Password Resets \nMaintaining the security of your accounts is crucial. If you forget your password or need to reset \nit, follow the steps below to securely regain access to your account. \n3.1 Using the Self-Service Password Reset (SSPR) Portal \nOur organization provides a Self-Service Password Reset (SSPR) portal that allows you to reset \nyour password without contacting the IT Help Desk. \nPrerequisites: \n• Ensure you are enrolled in the SSPR system. Enrollment involves setting up \nauthentication methods (e.g., phone number, email) that will be used to verify your \nidentity during the password reset process. 

In [ ]:
doc_splits[1].page_content

'o Enter your new password in both the "New password" and "Confirm new \npassword" fields. \no Click Next to finalize the reset. \n6. Confirmation: \no A confirmation message will indicate that your password has been successfully \nreset. \no You can now use your new password to log in to your account. \n3.2 Best Practices for Password Security \nTo maintain the security of your account, adhere to the following best practices: \n• Create Strong Passwords: \no Use a combination of uppercase and lowercase letters, numbers, and special \ncharacters. \no Avoid using easily guessable information such as birthdays, common words, or \nsequences (e.g., "password123" or "abcd1234"). \n• Unique Passwords: \no Do not reuse passwords across multiple accounts. Each account should have a \nunique password to prevent a compromise of one account leading to others being \ncompromised. \n• Regular Updates: \no Change your password periodically, at least every 90 days, to enhance security. \n• Use Multi-

In [ ]:
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain.vectorstores import Chroma


vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="guidelines",
    embedding = OpenAIEmbeddings(),
    persist_directory = "/content/drive/MyDrive/helpdesk_usecase/knowledgebase",

)
vectorstore.persist()

<ipython-input-12-2296f2d5c43d>:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

vec_db = Chroma(persist_directory=r"/content/drive/MyDrive/helpdesk_usecase/knowledgebase",
                embedding_function=embedding, collection_name="guidelines")

<ipython-input-14-abceff2e3443>:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vec_db = Chroma(persist_directory=r"/content/drive/MyDrive/helpdesk_usecase/knowledgebase",


In [ ]:
retriever = vec_db.as_retriever(search_kwargs = {'k':10})

## Multi-Agent workflow - LANGGRAPH

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_IT_issues_resolution_instructions",
    "Search and return guidelines to resolve the IT user request.",
)

resolution_tool = [retriever_tool]

In [ ]:
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from typing import Annotated, Literal, Sequence, TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


def query_transformation(state):

  messages = state["messages"]
  query = messages[-1].content
  llm = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")

  prompt = ChatPromptTemplate.from_messages([
    ("system",
     """ You are an expert in interpreting and converting user queries into precise IT support requests,
     your task is to analyze the given user query and generate a clear, concise, and actionable IT support request optimized
     to be interpretted by IT Support Expert.​

        Instructions:
        - Retain all technical terms, acronyms, and specific details provided by the user without attempting to rephrase them.​
        - Focus on extracting the core issue from the user's query to formulate a request that directly addresses their concern.​

     """),
    ("human", "{query}")
  ])

  chain = prompt | llm | StrOutputParser()
  transformed_query = chain.invoke({"query": query})



  return {"messages": [transformed_query]}


In [ ]:
from langchain_core.prompts import PromptTemplate

class classify(BaseModel):
        """Classification of complex queries."""
        resolvable: str = Field(description="Complexity Score 'yes' or 'no'")

def router(state)-> Literal["resolution_agent", "esclation_agent"]:

    prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
      You are an expert in IT support request classification, assess the complexity and severity of each user-submitted IT request.
      Utilize the provided criteria to determine whether the issue can be resolved by the user through available resources or necessitates escalation to IT support.

      ##Output Instructions##:
      - return "yes" if the IT request is user-resolvable else return "no" if escalation is required


      **Criteria for Self-Service Resolution (User-Resolved)**:
      Users can address certain issues independently by utilizing available resources such as knowledge bases, FAQs, or user manuals. These typically include:​
      - Password Resets: Users can follow predefined steps to reset their passwords.​
      - Software Installations: Guided instructions for installing approved software applications.​
      - Basic Troubleshooting: Steps for resolving common issues like printer connectivity problems or setting up email accounts.​

      **Criteria for IT Support Intervention and Escalation:
      When issues surpass the user's ability to resolve them, IT support intervention is necessary. Criteria for escalating a request to IT support include:​
      - Technical Complexity: Problems requiring specialized technical skills beyond the user's knowledge, such as network configuration issues or hardware malfunctions.​
      - High Impact on Business Operations: Incidents causing significant disruption to business processes, affecting multiple users or critical systems.​
      - Data Loss or Security Concerns: Situations involving potential data loss, security breaches, or compliance issues.​
      - Repeated Incidents: Persistent issues that recur despite previous troubleshooting efforts.​
      - Service Level Agreement (SLA) Breach: When an incident is approaching or has surpassed the agreed-upon response or resolution timeframes.

     """),
    ("human", "{query}")
  ])

    model = ChatOpenAI(temperature=0, model="gpt-4o", streaming=True)
    llm_with_tool = model.with_structured_output(classify)
    chain = prompt | llm_with_tool

    messages = state["messages"]
    query = messages[1].content


    result = chain.invoke({"query": query})

    classification = result.resolvable


    if classification == "yes":
        return "resolution_agent"
    else:
        return "esclation_agent"



In [ ]:
def resolution_agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end."""

    messages = state["messages"]
    model = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")
    model = model.bind_tools(resolution_tool)
    response = model.invoke(messages)

    return {"messages": [response]}

In [ ]:
def check_resolvable(state)-> Literal["resolution_agent", "esclation_agent"]:


  prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are an IT support expert tasked with evaluating whether a user's request can be resolved using the provided resolution response.

    Instructions:
    1. Review the user's query and the corresponding resolution response.
    2. Determine if the resolution response contains sufficient and clear instructions for the user to resolve the issue independently.
    3. If the resolution is adequate for self-resolution, return "yes".
    4. If the resolution lacks sufficient detail or is too complex for self-resolution, return "no".

    Please provide your assessment as either "yes" or "no" without additional commentary.
        """),
        ("human", "Query: {query}\n\nResolution Response: {resolution_response}")
  ])


  model = ChatOpenAI(temperature=0, model="gpt-4o", streaming=True)
  llm_with_tool = model.with_structured_output(classify)
  chain = prompt | llm_with_tool

  query = state["messages"][1].content
  resolution_response = state["messages"][-1].content


  result = chain.invoke({"query": query, "resolution_response": resolution_response})

  classification = result.resolvable

  if classification == "yes":
      return "resolution_agent"
  else:
      return "esclation_agent"

In [ ]:
def reranker(state):

  query = state["messages"][1].content
  documents = state["messages"][-1].content

  llm = ChatOpenAI(temperature=0, streaming=True, model="gpt-4o")

  prompt = ChatPromptTemplate.from_messages([
    ("system",
     """ You are an expert in information retrieval.
     Your task is to evaluate the relevance of each document provided below in relation to the user's query.
     After assessing all documents, identify and list the top 5 most relevant documents in descending order of relevance.


     """),
    ("human", "Query: {query} \n\n Documents: {documents}")
  ])

  chain = prompt | llm | StrOutputParser()
  reranked_documents = chain.invoke({"query": query, "documents":documents })

  return {"messages": [reranked_documents]}


In [ ]:
def generate(state):
    """
    Generate a resolution using context

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    messages = state["messages"]
    query = messages[1].content # transformed query
    last_message = messages[-1]

    docs = last_message.content

    prompt = ChatPromptTemplate.from_messages([
    ("system",
     """ You are an IT request Resolution assistant. Use the following pieces of retrieved context to answer the user query. If you don't know the answer, just say that you don't know.

        Instructions:
        - Devise a clear resolution for the IT request.
        - Detail all the steps clearly for resolving the user query.
        - Generate a well-formulated and structured answer for the user request.
        - Donot add extra information in the query, be specific.
     """),

    ("human", "User Query: {query} \n Context:{context}")
    ])

    # LLM
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0, streaming=True)

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    response = rag_chain.invoke({"context": docs, "query": query})
    return {"messages": [response]}


In [ ]:
from langgraph.types import Command, interrupt

# human in the loop
def esclation_agent(state):

    form = """
      **IT Request Form:**

      Incident Description
      [Provide a brief overview of the problem encountered]​

      System Information
      [Device Type, Operating System, Software, Hardware]

      Troubleshooting Steps Taken
      [List any troubleshooting steps you've already tried]​

    """
    filled_form = interrupt(form)

    return {"messages": [filled_form]}

In [ ]:
from langchain.chains import create_extraction_chain
import json

class Request_Details(BaseModel):
        IncidentDescription: str = Field(description="Description of Incident")
        SystemInformation: str = Field(description = "Information of the System")
        TroubleShootingStesps: str = Field(description="Steps taken for Intial Resolution")


def extract_additional_details(state):
  form = state["messages"][-1].content

  llm = ChatOpenAI(model="gpt-4o", temperature = 0.5)

  model_with_structure = llm.with_structured_output(Request_Details)

  details = model_with_structure.invoke(form)

  return {"messages", ["Esclation Report \n\n" + details.dict()]}


In [ ]:

class Request_Details(BaseModel):
        Esclation_Level: str = Field(description= "Level of esclation")
        Esclation_Justification: str = Field(description = "Justification of Esclation")
        IncidentDescription: str = Field(description="Description of Incident")
        SystemInformation: str = Field(description = "Information of the System")
        TroubleShootingStesps: str = Field(description="Steps taken for Intial Resolution")

def generate_structured_summary(state):


  query = state["messages"][1].content
  details = state["messages"][-1].content

  prompt = ChatPromptTemplate.from_messages([
    ("system",
     """ You are an IT Support escalation reporting assistant. Your task is to generate a structured esclation summary for the User IT Request.

      Instructions:
      - Summary must include the information from the user query and Additional user request.
      - Follow the template to generate a structured esclation summary.
      - Create a formal IT escalation summary to be submitted to IT Support for further resolution.

      **Summary template:**
        - Esclation Level:
        - Reason for Esclation:
        - System Information:
        - Trouble Shooting Steps:

     """),
    ("human", "Initial User Query: {query} Additional User Request Details: {request}")
  ])


  model = ChatOpenAI(temperature=0.5, model="gpt-4o", streaming=True)
  # chain = prompt | model

  model_with_structure = model.with_structured_output(Request_Details)

  summary = model_with_structure.invoke(query + "n\n" + details)

  summary = "Esclation Report\n\n" + str(summary)

  return {"messages": [summary]}


Create Graph

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver


memory = MemorySaver()


workflow = StateGraph(AgentState)

#Nodes
workflow.add_node("query_transformation", query_transformation)
workflow.add_node("resolution_agent", resolution_agent)
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)
workflow.add_node("reranker", reranker)
workflow.add_node("generate", generate)
workflow.add_node("esclation_agent", esclation_agent)
workflow.add_node("generate_structured_summary", generate_structured_summary)


workflow.add_edge(START, "query_transformation")

# Conditional Edge for routing queries to agent
workflow.add_conditional_edges(
    "query_transformation",
    router,
)

# Conditional Edge for resolution retrieval
workflow.add_conditional_edges(
    "resolution_agent",
    tools_condition,
    {
        "tools": "retrieve",
        END: END,
    },
)

# Conditional Edge for esclating queries to esclation agent if not resolvable
workflow.add_conditional_edges(
    "generate",
    check_resolvable,
)

workflow.add_edge("retrieve","reranker")
workflow.add_edge("reranker","generate")
workflow.add_edge("esclation_agent","generate_structured_summary")


# workflow.add_edge("generate", "resolution_agent")
workflow.add_edge("generate_structured_summary", "esclation_agent")
workflow.add_edge("esclation_agent", END)
workflow.add_edge("resolution_agent",END)


graph = workflow.compile(checkpointer=memory)


**Resolution without Esclation Case**

In [ ]:

config = {"configurable": {"thread_id": "1"}}

for s in graph.stream(

    {"messages": [HumanMessage(
        content="How to reset my password")]},
    config
):
    if "__end__" not in s:
        print(s)
        print("-----")

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1624: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'query_transformation': {'messages': ['Request: User needs assistance with resetting their password. Please provide step-by-step instructions or initiate a password reset process for the user.']}}
-----
{'resolution_agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_slltpcykQ3m3Bdw7sMKwj3kV', 'function': {'arguments': '{"query":"reset password"}', 'name': 'retrieve_IT_issues_resolution_instructions'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_432e014d75'}, id='run-331f73b3-e5fa-413d-a1a8-17365d692d18-0', tool_calls=[{'name': 'retrieve_IT_issues_resolution_instructions', 'args': {'query': 'reset password'}, 'id': 'call_slltpcykQ3m3Bdw7sMKwj3kV', 'type': 'tool_call'}])]}}
-----
{'retrieve': {'messages': [ToolMessage(content='o Enter your new password in both the "New password" and "Confirm new \npassword" fields. \no Click Next to finalize the r

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1624: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'generate': {'messages': ['To assist you with resetting your password, please follow these step-by-step instructions using the Self-Service Password Reset (SSPR) portal:\n\n1. **Access the SSPR Portal:**\n   - Open your web browser and navigate to the Self-Service Password Reset (SSPR) portal. The URL is typically provided by your organization, so please check your internal resources or contact your IT department if you are unsure.\n\n2. **Enter User Information:**\n   - On the SSPR portal homepage, you will be prompted to enter your user information. This usually includes your username or email address associated with your account.\n\n3. **Select the Issue Type:**\n   - Choose the option that indicates you need to reset your password. This is often labeled as "Forgot Password" or "Reset Password."\n\n4. **Verify Your Identity:**\n   - Follow the on-screen instructions to verify your identity. This may involve answering security questions, receiving a verification code via email or SM

In [ ]:
graph.get_state(config).values["messages"][-1].content

'To assist you with resetting your password, please follow these step-by-step instructions using the Self-Service Password Reset (SSPR) portal:\n\n1. **Access the SSPR Portal:**\n   - Open your web browser and navigate to the Self-Service Password Reset (SSPR) portal at [https://aka.ms/sspr](https://aka.ms/sspr).\n\n2. **Enter User Information:**\n   - Enter your full company email address in the provided field.\n   - Complete the CAPTCHA verification to confirm you are not a robot.\n   - Click "Next" to proceed.\n\n3. **Select the Issue Type:**\n   - Choose "I forgot my password" from the available options.\n   - Click "Next."\n\n4. **Verify Your Identity:**\n   - Select your preferred verification method from the options you configured during enrollment (e.g., text message, email).\n   - Follow the on-screen instructions to complete the verification process, which may involve entering a code sent to your phone or email.\n\n5. **Reset Your Password:**\n   - Once your identity is verif

**Example Run for Esclation User Query**




The query is routed to an escalation agent, who interrupts to gather human feedback. The user shares system information and troubleshooting details. An Escalation Summary is then generated using a predefined schema, including Escalation Level, Justification, Incident Description, System Information, and Troubleshooting Steps.

In [ ]:
config = {"configurable": {"thread_id": "2"}}

for s in graph.stream(

    {"messages": [HumanMessage(
        content="My VPN is not working."
        )]},
    config
):
    if "__end__" not in s:
        print(s)
        print("-----")

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1624: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


{'query_transformation': {'messages': ['Request: Investigate and resolve the issue with the VPN not functioning.']}}
-----
{'__interrupt__': (Interrupt(value="\n      **IT Request Form:**\n\n      Incident Description\n      [Provide a brief overview of the problem encountered]\u200b\n\n      System Information\n      [Device Type, Operating System, Software, Hardware]\n\n      Troubleshooting Steps Taken\n      [List any troubleshooting steps you've already tried]\u200b\n\n    ", resumable=True, ns=['esclation_agent:5e1dbeb1-b971-6e1a-4d87-fb0a1e4c6912']),)}
-----


In [ ]:
human_feedback = """
I am experiencing issues with my VPN connection; it is not functioning as expected.​

System Information:

Device Type: Laptop​

Operating System: Windows 10 Pro​

VPN Software: Cisco AnyConnect Secure Mobility Client​

Hardware: HP EliteBook 840 G7, Intel Core i7, 16 GB RAM​

Troubleshooting Steps Taken:
I have taken several steps to resolve the VPN connectivity issue.
Initially, I ensured that my internet connection is stable by verifying network connectivity.
I then restarted the Cisco AnyConnect application to reset the connection, followed by rebooting my HP EliteBook to address any temporary system glitches.
I also confirmed that my VPN credentials are correct and up to date.
Additionally, I reviewed firewall and security software settings to ensure they are not interfering with the VPN connection.
I attempted to connect to different VPN servers to rule out server-specific issues and made sure that the Cisco AnyConnect application is updated to the latest version.
I also verified that the correct VPN protocol is selected and that the VPN configuration aligns with recommended settings.
Finally, I consulted the Cisco AnyConnect support resources, including troubleshooting guides and FAQs.
Despite these efforts, the VPN connection issue persists. I would appreciate further assistance in resolving this matter.
"""

for event in graph.stream(Command(resume = human_feedback), config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Request: Investigate and resolve the issue with the VPN not functioning.
================================ Human Message =================================


I am experiencing issues with my VPN connection; it is not functioning as expected.​

System Information:

Device Type: Laptop​

Operating System: Windows 10 Pro​

VPN Software: Cisco AnyConnect Secure Mobility Client​

Hardware: HP EliteBook 840 G7, Intel Core i7, 16 GB RAM​

Troubleshooting Steps Taken: 
I have taken several steps to resolve the VPN connectivity issue. 
Initially, I ensured that my internet connection is stable by verifying network connectivity. 
I then restarted the Cisco AnyConnect application to reset the connection, followed by rebooting my HP EliteBook to address any temporary system glitches. 
I also confirmed that my VPN credentials are correct and up to date. 
Additionally, I reviewed firewall and security software settings t

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1624: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


================================ Human Message =================================

Esclation Report

Esclation_Level='High' Esclation_Justification='The VPN connectivity issue persists despite extensive troubleshooting efforts, impacting work productivity.' IncidentDescription='The VPN connection using Cisco AnyConnect Secure Mobility Client is not functioning as expected on a Windows 10 Pro laptop (HP EliteBook 840 G7).' SystemInformation='Device Type: Laptop, Operating System: Windows 10 Pro, VPN Software: Cisco AnyConnect Secure Mobility Client, Hardware: HP EliteBook 840 G7, Intel Core i7, 16 GB RAM' TroubleShootingStesps='1. Verified stable internet connection. 2. Restarted Cisco AnyConnect application. 3. Rebooted laptop. 4. Confirmed VPN credentials. 5. Reviewed firewall and security software settings. 6. Attempted connection to different VPN servers. 7. Updated Cisco AnyConnect to latest version. 8. Verified correct VPN protocol and configuration. 9. Consulted Cisco AnyConnect s

In [ ]:
graph.get_state(config).values["messages"][-1].content

"Esclation Report\n\nEsclation_Level='High' Esclation_Justification='The VPN connectivity issue persists despite extensive troubleshooting efforts, impacting work productivity.' IncidentDescription='The VPN connection using Cisco AnyConnect Secure Mobility Client is not functioning as expected on a Windows 10 Pro laptop (HP EliteBook 840 G7).' SystemInformation='Device Type: Laptop, Operating System: Windows 10 Pro, VPN Software: Cisco AnyConnect Secure Mobility Client, Hardware: HP EliteBook 840 G7, Intel Core i7, 16 GB RAM' TroubleShootingStesps='1. Verified stable internet connection. 2. Restarted Cisco AnyConnect application. 3. Rebooted laptop. 4. Confirmed VPN credentials. 5. Reviewed firewall and security software settings. 6. Attempted connection to different VPN servers. 7. Updated Cisco AnyConnect to latest version. 8. Verified correct VPN protocol and configuration. 9. Consulted Cisco AnyConnect support resources.'"